In [6]:
#Initialization for HateSpeechRecognition
import scipy as sp
import numpy as np
import pandas as pd
import sklearn
from sklearn import svm
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /home/mufa/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/mufa/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/mufa/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [7]:

def clean_text(rgx_list, text):
    new_text = text
    for rgx_match in rgx_list:
        new_text = re.sub(rgx_match, '', new_text)
    return new_text

In [8]:

stopwords_list = stopwords.words('english')
#other_exclusions = ["#ff", "ff", "rt","!",":","...",".","-","&","?"]
#stopwords_list.extend(other_exclusions)

dataset = pd.read_csv("dataset/labeled_data.csv")

tweets = dataset.tweet

ps = nltk.stem.PorterStemmer()

#ps.stem(dataset)

#Text cleaning and tokenization, then stemming then POS tagging
filtered_tweets=[];
tweet_tags = [];
filtered_tweets_stemmed=[];

for tweet in tweets:
    tweet = clean_text(["http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+",".*@.*:","&#*\w*","@[\w\-]+","[^\w\s]"],tweet)
    tweet = tweet.lower()
    word_tokens = word_tokenize(tweet)
    filtered_tweets.append([word for word in word_tokens if not word in stopwords_list])
    filtered_tweets_stemmed.append([ps.stem(word) for word in word_tokens if not word in stopwords_list])
    
for tweet in filtered_tweets:
    tweet_tags.append(nltk.pos_tag(tweet))

    

In [35]:
#Creating TfIdf vectorizer
ftss=[]
for tweet in filtered_tweets_stemmed:
    ftss.append(' '.join(tweet))
clf = svm.SVC(decision_function_shape='ovo')
vectorizer = sklearn.feature_extraction.text.TfidfVectorizer(ngram_range=(1,3),lowercase=False,max_features=50000)
X = vectorizer.fit_transform(ftss)

In [58]:
Y = dataset['class']

clf.fit(X,Y)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovo', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [60]:
clf.predict(X[1])

array([1])